In [2]:
from bs4 import BeautifulSoup
import mechanicalsoup
import requests
import time, os
from selenium import webdriver
import math
import pickle
import re
import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tummee.com/login?")

cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://www.tummee.com/sequences/teachers/daily")

In [4]:
f = open("yin_urls", "rb")
yin_urls = pickle.load(f)
f.close()

In [10]:
def get_poses(url): 
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try: 
        title = re.sub(r"[\t\n0-9.]", "", soup.find("h1").text.strip())
        find_poses = soup.find_all("a", class_="english-title")
        poses = [re.sub(r"[\t\n0-9.]", "", i.text.strip()) for i in find_poses]
        class_type = "Yin"
    except AttributeError:
        info = {"Title": np.nan}
    except TimeoutError:
        info = {"Title": np.nan}
    else: 
        info = {"Title": title, "Poses": poses, "Class Type": class_type}
    return info

In [6]:
len(yin_urls)

4587

In [8]:
yin_classes_1 = [get_poses(url) for url in yin_urls[0:1000]]

In [29]:
df = pd.DataFrame(yin_classes_1)

In [11]:
yin_classes_2 = [get_poses(url) for url in yin_urls[1000:3000]]

In [30]:
df2 = pd.DataFrame(yin_classes_2)

In [23]:
yin_classes_3 = [get_poses(url) for url in yin_urls[3000:]]

In [31]:
df3 = pd.DataFrame(yin_classes_3)

In [32]:
yin_df = pd.concat([df, df2, df3])

In [33]:
yin_df

,Title,Poses,Class Type
0,←Yin chair,"[Chair Neck Rolls B, Shoulders Lift And Drop C...",Yin
1,←Moon Hatha/Yin Hybrid July,[Easy Pose Fingers On Floor Arms Raised Overhe...,Yin
2,←Letting go,"[Easy Pose, Butterfly Pose Variation Forward B...",Yin
3,←Yin,"[Centering, Easy Pose Block, Pranayama, Ujjayi...",Yin
4,←YIN PSOAS,"[Corpse Pose, Half Wind Release Pose, Supine S...",Yin
...,...,...,...
1582,←Shoulder-Heart Opener,"[Easy Pose, Easy Pose Neck Side Stretch, Easy ...",Yin
1583,←Yin Sequence,"[Easy Pose, Cat Cow Pose, Child Pose, Seated S...",Yin
1584,←Practicing Presence,"[Easy Pose Hands To Heart, Easy Pose Twist Rai...",Yin
1585,←Yin- Cooling,"[Sitali, Bound Angle Pose Blocks, Sun Breaths,...",Yin


In [34]:
f = open("yin_df", "wb")
pickle.dump(yin_df, f)
f.close()